In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from nltk.corpus import gutenberg, stopwords
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [2]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    text = re.sub(r'CHAPTER \d+', '', text)
    text = re.sub("\\n\\n.*?\\n\\n", '', text)
  
    # Get rid of extra whitespace.
    text = ' '.join(text.split())

    
    return text

In [3]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

## Data Cleaning & Setup

In [4]:
# load documents
ball = gutenberg.raw('chesterton-ball.txt')
brown = gutenberg.raw('chesterton-brown.txt')
thursday = gutenberg.raw('chesterton-thursday.txt')

# clean texts
ball = text_cleaner(ball)
brown = text_cleaner(brown)
thursday = text_cleaner(thursday)

# run spacy and analyze the documents
nlp = spacy.load('en')
ball_doc = nlp(ball)
brown_doc = nlp(brown)
thursday_doc = nlp(thursday)

In [5]:
brown_sents = []
ball_sents = []
thursday_sents = []
for sentence in brown_doc.sents:
    brown_sents.append(sentence)
for sentence in ball_doc.sents:
    ball_sents.append(sentence)
for sentence in thursday_doc.sents:
    thursday_sents.append(sentence)

In [6]:
print('Length of\nBrown sentences: {}\nBall sentences: {}\nThursday sentences: {}'.format(len(brown_sents), len(ball_sents), len(thursday_sents)))

Length of
Brown sentences: 3534
Ball sentences: 4272
Thursday sentences: 3053


In [7]:
brown_sents = brown_sents[:2000]
ball_sents = ball_sents[:2000]
thursday_sents = thursday_sents[:2000]

In [8]:
# combine the documents into a dataframe with identifier
ball_df = [[sent, "Ball"] for sent in ball_sents]
brown_df = [[sent, "Brown"] for sent in brown_sents]
thursday_df = [[sent, "Thursday"] for sent in thursday_sents]

sentences = pd.DataFrame(ball_df + brown_df + thursday_df)
sentences.head()

,0,1
0,"(The, flying, ship, of, Professor, Lucifer, sa...",Ball
1,"(That, it, was, far, above, the, earth, was, n...",Ball
2,"(The, professor, had, himself, invented, the, ...",Ball
3,"(Every, sort, of, tool, or, apparatus, had, ,,...",Ball
4,"(For, the, world, of, science, and, evolution,...",Ball


In [9]:
len(sentences)

6000

## Bag of Words

In [10]:
# Utility function to create a list of the 750 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(750)]

# Utility function to remove words from document that aren't in truncated sentence list
def remove_missing_words(words, sentences):
    text = ''
    for token in sentences:
        text += token.text
        
    for word in words:
        if word not in text:
            words.remove(word)
    return words
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    print('Setting up the Dataframe...')
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 250 == 0:
            print("Processing row {}".format(i))
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
    
    return df

In [11]:
# Set up the bags.
ball_words = bag_of_words(ball_doc)
thursday_words = bag_of_words(thursday_doc)
brown_words = bag_of_words(brown_doc)

# Remove missing common words
ball_words = remove_missing_words(ball_words, ball_sents)
brown_words = remove_missing_words(brown_words, brown_sents)
thursday_words = remove_missing_words(thursday_words, thursday_sents)

# Combine bags to create a set of unique words.
common_words = set(ball_words + thursday_words + brown_words)
len(common_words)

1120

In [12]:
# Create our data frame with features
word_counts = bow_features(sentences, common_words)
word_counts.head()

Setting up the Dataframe...
Processing row 0
Processing row 250
Processing row 500
Processing row 750
Processing row 1000
Processing row 1250
Processing row 1500
Processing row 1750
Processing row 2000
Processing row 2250
Processing row 2500
Processing row 2750
Processing row 3000
Processing row 3250
Processing row 3500
Processing row 3750
Processing row 4000
Processing row 4250
Processing row 4500
Processing row 4750
Processing row 5000
Processing row 5250
Processing row 5500
Processing row 5750


,or,true,crawl,observe,person,have,carry,longer,spot,end,...,present,certainly,shock,council,detective,moon,path,name,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, flying, ship, of, Professor, Lucifer, sa...",Ball
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(That, it, was, far, above, the, earth, was, n...",Ball
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, professor, had, himself, invented, the, ...",Ball
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Every, sort, of, tool, or, apparatus, had, ,,...",Ball
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(For, the, world, of, science, and, evolution,...",Ball


In [39]:
# random forest fitting
rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9563888888888888

Test set score: 0.5154166666666666


In [44]:
cross_val_score(rfc, X, Y, cv=5)

array([0.445     , 0.48916667, 0.48666667, 0.48333333, 0.41916667])

In [40]:
# logistic regression fitting
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3600, 1120) (3600,)
Training set score: 0.7708333333333334

Test set score: 0.5891666666666666


In [43]:
cross_val_score(lr, X, Y, cv=5)

array([0.48666667, 0.51083333, 0.57666667, 0.52833333, 0.45833333])

In [41]:
# gradient boosting fitting
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.6488888888888888

Test set score: 0.5395833333333333


In [42]:
cross_val_score(clf, X, Y, cv=5)

array([0.46583333, 0.4725    , 0.51666667, 0.53416667, 0.44916667])

## Tf-idf

In [16]:
# tf-idf function
def document_freq(data, sentences, common_words, doc_names, doc_words):
    
    # initialize df
    df = pd.DataFrame(columns = common_words)
    df.iloc[:, 0] = [0, 0, 0, 0, 0, 0]
    df.loc[:, common_words] = 0
    df.rename(index={0:'df', 1:'cf', 2:'idf', 3:'ball', 4:'brown', 5:'thursday'}, inplace=True)
    
    for word in common_words:
#         # drop words that aren't existing; problem due to the way common words are set up
#         if data.loc[:, word].sum() == 0:
#             df.drop([word], axis=1, inplace=True)
#             continue
        
        # find document frequency & collection frequency
        df.loc['df', word] = data[data[word] > 0][word].count()
        df.loc['cf', word] = data.loc[:, word].sum()
        
        # find idf
        df.loc['idf', word] = np.log2(len(sentences)/df.loc['df', word])
        
    # assign the idf value to the documents
    for word in df.columns:
        for i in range(len(doc_names)):
            if word in doc_words[i]:
                df.loc[doc_names[i], word] = df.loc['idf', word]
        
    return df

In [27]:
doc_names = ['ball', 'brown', 'thursday']
doc_words = [ball_words, brown_words, thursday_words]
tf_idf = document_freq(word_counts, sentences, common_words, doc_names, doc_words)
tf_idf

c:\users\brian\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in longlong_scalars


,or,true,crawl,observe,person,have,carry,longer,spot,end,...,irregular,scarf,present,certainly,shock,council,detective,moon,path,name
df,12.000000,28.000000,11.000000,13.000000,36.000000,87.000000,23.000000,6.000000,11.000000,85.000000,...,7.000000,6.000000,21.000000,36.000000,16.000000,17.000000,25.000000,17.000000,11.000000,18.000000
cf,12.000000,28.000000,12.000000,14.000000,36.000000,90.000000,25.000000,7.000000,12.000000,87.000000,...,7.000000,6.000000,21.000000,38.000000,17.000000,17.000000,26.000000,18.000000,12.000000,18.000000
idf,8.965784,7.743392,9.091315,8.850307,7.380822,6.107803,8.027185,9.965784,9.091315,6.141356,...,9.743392,9.965784,8.158429,7.380822,8.550747,8.463284,7.906891,8.463284,9.091315,8.380822
ball,8.965784,7.743392,0.000000,0.000000,7.380822,6.107803,8.027185,9.965784,0.000000,6.141356,...,0.000000,0.000000,0.000000,7.380822,8.550747,0.000000,0.000000,8.463284,9.091315,8.380822
brown,0.000000,7.743392,9.091315,8.850307,7.380822,6.107803,8.027185,9.965784,9.091315,6.141356,...,0.000000,9.965784,8.158429,7.380822,8.550747,0.000000,7.906891,8.463284,9.091315,8.380822
thursday,0.000000,7.743392,0.000000,0.000000,7.380822,6.107803,8.027185,0.000000,0.000000,6.141356,...,9.743392,0.000000,8.158429,7.380822,0.000000,8.463284,7.906891,8.463284,0.000000,8.380822


In [28]:
tf_idf = tf_idf.T
tf_idf.head()

,df,cf,idf,ball,brown,thursday
or,12.0,12.0,8.965784,8.965784,0.000000,0.000000
true,28.0,28.0,7.743392,7.743392,7.743392,7.743392
crawl,11.0,12.0,9.091315,0.000000,9.091315,0.000000
observe,13.0,14.0,8.850307,0.000000,8.850307,0.000000
person,36.0,36.0,7.380822,7.380822,7.380822,7.380822


In [29]:
# set threshold & initialize
threshold = 5
tf_idf['ball_threshold'] = 0
tf_idf['brown_threshold'] = 0
tf_idf['thursday_threshold'] = 0

tf_idf['ball_threshold'] = np.where(tf_idf['ball'] > threshold, 1, 0)
tf_idf['brown_threshold'] = np.where(tf_idf['brown'] > threshold, 1, 0)
tf_idf['thursday_threshold'] = np.where(tf_idf['thursday'] > threshold, 1, 0)

tf_idf.head()

,df,cf,idf,ball,brown,thursday,ball_threshold,brown_threshold,thursday_threshold
or,12.0,12.0,8.965784,8.965784,0.000000,0.000000,1,0,0
true,28.0,28.0,7.743392,7.743392,7.743392,7.743392,1,1,1
crawl,11.0,12.0,9.091315,0.000000,9.091315,0.000000,0,1,0
observe,13.0,14.0,8.850307,0.000000,8.850307,0.000000,0,1,0
person,36.0,36.0,7.380822,7.380822,7.380822,7.380822,1,1,1


In [30]:
tf_idf.drop(['df', 'cf', 'idf', 'ball', 'brown', 'thursday'], axis=1, inplace=True)
tf_idf.head()

,ball_threshold,brown_threshold,thursday_threshold
or,1,0,0
true,1,1,1
crawl,0,1,0
observe,0,1,0
person,1,1,1


In [31]:
tf_idf['source'] = 'multiple'
tf_idf.head()

,ball_threshold,brown_threshold,thursday_threshold,source
or,1,0,0,multiple
true,1,1,1,multiple
crawl,0,1,0,multiple
observe,0,1,0,multiple
person,1,1,1,multiple


In [23]:
# determine which word belongs to which document
def set_doc(df):
    
    
    
    # loop that iterates through each row and determine what the source is
    for i in range(len(df)):
        
        #initialize
        flag = 0
        set_source = 'multiple'
    
        if (df.iloc[i, 0] == 1):
            flag = 1
            set_source = 'Ball'
            
        if (df.iloc[i, 1] == 1):
            if (flag == 1): 
                continue
            flag = 1
            set_source = 'Brown'
            
        if df.iloc[i, 2] == 1:
            if flag == 1:
                continue
            flag = 1
            set_source = 'Thursday'
            
        df.iloc[i, 3] = set_source
        
    return df

In [33]:
tf_idf = set_doc(tf_idf)
tf_idf.head(10)

,ball_threshold,brown_threshold,thursday_threshold,source
or,1,0,0,Ball
true,1,1,1,multiple
crawl,0,1,0,Brown
observe,0,1,0,Brown
person,1,1,1,multiple
have,1,1,1,multiple
carry,1,1,1,multiple
longer,1,1,0,multiple
spot,0,1,0,Brown
end,1,1,1,multiple


In [45]:
# random forest fitting
rfc = ensemble.RandomForestClassifier()
Y = tf_idf['source']
X = tf_idf.drop(['source'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.5)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 1.0

Test set score: 1.0


In [37]:
cross_val_score(rfc, X, Y, cv=5)

array([1., 1., 1., 1., 1.])

In [35]:
# logistic regression fitting
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(560, 3) (560,)
Training set score: 0.9857142857142858

Test set score: 0.9875


In [38]:
cross_val_score(lr, X, Y, cv=5)

array([0.99111111, 0.98660714, 0.99107143, 0.98660714, 0.97757848])

In [46]:
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 1.0

Test set score: 1.0


In [47]:
cross_val_score(lr, X, Y, cv=5)

array([0.99111111, 0.98660714, 0.99107143, 0.98660714, 0.97757848])